In [1]:
import numpy as np
import pandas as pd
# import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller

import matplotlib.pyplot as plt
# import seaborn as sns; sns.set(style="whitegrid")
import seaborn
import warnings
warnings.filterwarnings('ignore')

In [2]:
def find_cointegrated_pairs(date1, date2):
    df = pd.read_csv('cleanest_data_ever.csv', index_col = 'Date')
    data = df.loc[date1 : date2]
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.05:
                pairs.append([keys[i], keys[j]])
    return score_matrix, pvalue_matrix, pairs

In [3]:
# date format: year-month-day
def slicing_formation(date1, date2):
    df = pd.read_csv('cleanest_data_ever.csv', index_col = 'Date')
    dta = df.loc[date1:date2]
    return(dta)
def slicing_trading(dta1, dta2):
    df = pd.read_csv('cleanest_data_ever.csv', index_col = 'Date')
    dta = df.loc[dta1 : dta2]
    return(dta)
def normalizing(data): # input: sliced dataset, output: normalized dataset
    df = pd.DataFrame(index = data.index, columns = data.columns)
    for i in df.columns:
        df[i][0] = 1
        for j in range(1, len(df)):
            df[i][j] = data[i][j] / data[i][0]
    return(df)
def ssd(col1, col2):
    s = (col1 - col2)**2
    ssd = s.sum()
    return(ssd)
def ssd_matrix(dta): # input normalized dataset
    matrix = pd.DataFrame(index = dta.columns, columns = dta.columns)
    for i in matrix.index:
        for j in matrix.columns:
            if i != j:
                matrix.loc[i,j] = ssd(dta[i], dta[j])
    return(matrix)
def min_values(matrix): # input: ssd matrix
    minimums = []
    for i in matrix.index:
        for j in matrix.columns:
            minimums.append(matrix.loc[i,j])
    minimums.remove(minimums[0])
    minimums = list(dict.fromkeys(minimums))
    minimums.sort()
    
#     minimums_complete = []
#     while len(minimums_complete) < 10:
#         minimum = min(minimums)
#         minimums.remove(minimum)
#         minimums_complete.append(minimum)
#     min_set = set(minimums_complete)
#     list_min = list(min_set)
    return(minimums)
def standard_deviation(col1,col2):
    series = col1 - col2
    std = stat.stdev(series)
    return(std)

In [4]:
def forming_pairs(date1, date2):
    dataframe = slicing_formation(date1, date2)
    norm_dataframe = normalizing(dataframe)
    coef_matrix = ssd_matrix(norm_dataframe)
    coef_list = min_values(coef_matrix)
    return(coef_matrix, coef_list)
def pair_tickers(coeff_matrix, coeff_list):
    pair_names = []
    for c in coeff_list:
        for i in coeff_matrix.index:
            for j in coeff_matrix.columns:
                if coeff_matrix.loc[i,j] == c:
                    names = [i,j]
                    pair_names.append(names)
#     unique_pairs = []
#     for r in (0,2,4,6,8):
#         unique_pairs.append(pair_names[r])
    for i in pair_names:
        pair_names.remove([i[1], i[0]])
    return(pair_names)

In [5]:
def coint_pairs(date1, date2, x):
    sd_mat, sd_coef = forming_pairs(date1, date2)
    sd_pairs = pair_tickers(sd_mat, sd_coef)
    _, _, coint_pairs = find_cointegrated_pairs(date1, date2)
    pairs = []
    for i in sd_pairs:
        pair1 = i[0]
        pair2 = i[1]
        if [pair1, pair2] in coint_pairs or [pair2, pair1] in coint_pairs: 
            pairs.append(i)
        else:
            pass
    return(pairs[:x])
    


In [6]:
def spread_stat(date1, date2, pair1, pair2): # dates are for the formation period
    df = pd.read_csv('cleanest_data_ever.csv', index_col = 'Date')
    dta = df.loc[date1 : date2]
    data = dta.divide(dta.loc[date1])
    s1 = data[pair1]
    s2 = data[pair2]
    
    s1 = sm.add_constant(s1)
    results = sm.OLS(s2, s1).fit()
    s1 = s1[pair1]
    b = results.params[pair1]
    
    spread = s2 - b * s1
    std = spread.std()
    mean = spread.mean()
    return(std, mean)

In [7]:
def signal(date1, date2, std, mean, pair1, pair2):
    df = pd.read_csv('cleanest_data_ever.csv', index_col = 'Date')
    dta = df.loc[date1 : date2]
    data = dta.divide(dta.loc[date1])
    s1 = data[pair1]
    s2 = data[pair2]
    
    s1 = sm.add_constant(s1)
    results = sm.OLS(s2, s1).fit()
    s1 = s1[pair1]
    b = results.params[pair1]
    assert(len(s1) == len(s2))
    spread = s2 - b * s1
    norm_spread = (spread - mean) / std
    df1 = pd.DataFrame(index = s1.index, columns = ['Close', 'norm_spread'])
    df1['Close'] = dta[pair1]
    df1['norm_spread'] = norm_spread
    df2 = pd.DataFrame(index = s2.index, columns = ['Close', 'norm_spread'])
    df2['Close'] = dta[pair2]
    df2['norm_spread'] = norm_spread
    df1 = df1.reset_index()
    df2 = df2.reset_index()
    df1['Signal'] = pd.Series(np.zeros(len(df1)))
    df1['Signal'][0] = 0
    df2['Signal'] = pd.Series(np.zeros(len(df2)))
    df2['Signal'][0] = 0
    for x in range(1, len(df1)):
        if df1['norm_spread'][x] > 2:
            df1['Signal'][x] = 1
            df2['Signal'][x] = -1
        elif df1['norm_spread'][x] < -2:
            df1['Signal'][x] = -1
            df2['Signal'][x] = 1
        elif df1['norm_spread'][x] == 0:
            df1['Signal'][x] = 0
            df2['Signal'][x] = 0
        elif (df1['norm_spread'][x - 1]) / (df1['norm_spread'][x]) < 0:
            df1['Signal'][x] = 0
            df2['Signal'][x] = 0
        else:
            df1['Signal'][x] = df1['Signal'][x - 1]
            df2['Signal'][x] = df2['Signal'][x - 1]
    df1['Position'] = pd.Series(np.zeros(len(df1)))
    df1['Position'] = df1['Signal'].diff()
    df2['Position'] = pd.Series(np.zeros(len(df2)))
    df2['Position'] = df2['Signal'].diff()
    return(df1, df2)
    

In [77]:
def trading(dta):
    dta['Total Assets'] = pd.Series(np.zeros(len(dta)))
    dta['Holding'] = pd.Series(np.zeros(len(dta)))
    dta['Employed Capital'] = pd.Series(np.zeros(len(dta)))
    dta['Shorted Capital'] = pd.Series(np.zeros(len(dta)))
    dta['Free Capital'] = pd.Series(np.zeros(len(dta)))
    dta['Total Assets'][0] = 10000
    dta['Holding'][0] = 0
    dta['Employed Capital'][0] = 0
    dta['Shorted Capital'][0] = 0
    dta['Free Capital'][0] = 10000
    shorted_cap = 0
    for x in range(1, len(dta)):
        if dta['Position'][x] == 1:
            if dta['Holding'][x - 1] == 0:
                dta['Holding'][x] = dta['Total Assets'][x - 1] // dta['Close'][x]
                dta['Employed Capital'][x] = dta['Holding'][x] * dta['Close'][x]
                dta['Free Capital'][x] = dta['Total Assets'][x -1] - dta['Employed Capital'][x]
                dta['Total Assets'][x] = dta['Free Capital'][x] + dta['Employed Capital'][x]
            else:
                dta['Employed Capital'][x] = dta['Employed Capital'][x - 1]
                profit_loss = (dta['Holding'][x - 1] * dta['Close'][x]) - shorted_cap
                dta['Total Assets'][x] = dta['Free Capital'][x - 1] + dta['Employed Capital'][x] + profit_loss
                dta['Free Capital'][x] = dta['Total Assets'][x]
                dta['Holding'][x] = 0
                dta['Shorted Capital'][x] = 0
#                 print(profit_loss)
        elif dta['Position'][x] == -1:
            if dta['Holding'][x - 1] == 0:
                dta['Holding'][x] = -(dta['Total Assets'][x - 1] // dta['Close'][x])
                dta['Employed Capital'][x] = dta['Employed Capital'][x - 1]
                dta['Free Capital'][x] = dta['Free Capital'][x - 1]
                shorted_cap = dta['Holding'][x] * dta['Close'][x]
#                 print(shorted_cap)
                dta['Shorted Capital'][x] = dta['Holding'][x] * dta['Close'][x]
                dta['Total Assets'][x] = dta['Free Capital'][x] + dta['Employed Capital'][x]
            else:
                dta['Shorted Capital'][x] = dta['Shorted Capital'][x - 1]
                dta['Free Capital'][x] = dta['Free Capital'][x - 1]
                dta['Employed Capital'][x] = 0
                dta['Total Assets'][x] = dta['Free Capital'][x] + dta['Holding'][x - 1] * dta['Close'][x]
        else:
            dta['Holding'][x] = dta['Holding'][x - 1]
            if dta['Holding'][x] > 0:
                dta['Shorted Capital'][x] = dta['Shorted Capital'][x - 1]
                dta['Free Capital'][x] = dta['Free Capital'][x - 1]
                dta['Employed Capital'][x] = dta['Holding'][x] * dta['Close'][x]
                dta['Total Assets'][x] = dta['Free Capital'][x] + dta['Employed Capital'][x]
            elif dta['Holding'][x] < 0:
                dta['Employed Capital'][x] = dta['Employed Capital'][x - 1]
                dta['Free Capital'][x] = dta['Free Capital'][x - 1]
                dta['Shorted Capital'][x] = dta['Holding'][x] * dta['Close'][x]
                dta['Total Assets'][x] = dta['Free Capital'][x] + dta['Employed Capital'][x] 
            else:
                dta['Free Capital'][x] = dta['Free Capital'][x - 1]
                dta['Employed Capital'][x] = dta['Employed Capital'][x - 1]
                dta['Shorted Capital'][x] = dta['Shorted Capital'][x - 1]
                dta['Total Assets'][x] = dta['Total Assets'][x - 1]
    if dta['Holding'][len(dta) - 1] > 0:
        profit = (dta['Total Assets'][len(dta) - 1] / dta['Total Assets'][0] - 1) * 100
    elif dta['Holding'][len(dta) - 1] < 0:
        profit1 = dta['Shorted Capital'][len(dta) - 1] - shorted_cap
        profit = ((profit1 + dta['Total Assets'][len(dta) - 1]) / dta['Total Assets'][0] - 1) * 100
    else:
        profit = (dta['Total Assets'][len(dta) - 1] / dta['Total Assets'][0] - 1) * 100
    return(profit)

In [78]:
def Coint_Trading(date1, date2, date3, date4, x): # date1, date2 formation period, date3, date4 trading period, x stocks
    pairs = coint_pairs(date1, date2, x)
    returns = []
    for i in pairs:
        pair1 = i[0]
        pair2 = i[1]
        std, mean = spread_stat(date1, date2, pair1, pair2)
        df1, df2 = signal(date3, date4, std, mean, pair1, pair2)
        pair1_r = trading(df1)
        pair2_r = trading(df2)
        average_r = (pair1_r + pair2_r) / 2
        returns.append(average_r)
    return(returns)

In [79]:
def average(sequence): # input a list
    sum_ = sum(sequence)
    mean = sum_ / len(sequence)
    return(mean)

In [80]:
path = 'dates.csv' # monthly rotation
dates = pd.read_csv(path)

In [81]:
returns = pd.DataFrame(index = dates.index, columns = ['returns','date1','date2','date3','date4'])
for i in dates.index[45:]:
    date1 = dates.loc[i, 'date1']
    date2 = dates.loc[i, 'date2']
    date3 = dates.loc[i, 'date3']
    date4 = dates.loc[i, 'date4']
    seq = Coint_Trading(date1, date2, date3, date4, 20)
    value = average(seq)
    returns.loc[i, 'returns'] = value
    returns.loc[i, 'date1'] = dates.loc[i, 'date1']
    returns.loc[i, 'date2'] = dates.loc[i, 'date2']
    returns.loc[i, 'date3'] = dates.loc[i, 'date3']
    returns.loc[i, 'date4'] = dates.loc[i, 'date4']
    

In [82]:
returns.to_excel('top_20_pairs_returns_cointegration3.xlsx', index = True)

In [13]:
date1 = dates.loc[45, 'date1'] 
date2 = dates.loc[45, 'date2'] 
date3 = dates.loc[45, 'date3'] 
date4 = dates.loc[45, 'date4']

In [14]:
pairs = coint_pairs(date1, date2, 20)

In [15]:
pairs

[['BOKA.AS', 'OXUR.BR'],
 ['BAR.BR', 'SOF.BR'],
 ['AGS.BR', 'EXM.BR'],
 ['AGS.BR', 'RAND.AS'],
 ['BBED.AS', 'UCB.BR'],
 ['AGS.BR', 'VASTN.AS'],
 ['CFEB.BR', 'INGA.AS'],
 ['BBED.AS', 'TUB.BR'],
 ['ASML.AS', 'SOF.BR'],
 ['ASM.AS', 'EVS.BR'],
 ['BAMNB.AS', 'EXM.BR'],
 ['ASML.AS', 'DSM.AS'],
 ['ASM.AS', 'HEIJM.AS'],
 ['AGN.AS', 'OXUR.BR'],
 ['AALB.AS', 'TUB.BR'],
 ['BAR.BR', 'MELE.BR'],
 ['BOKA.AS', 'VASTN.AS'],
 ['AGN.AS', 'VASTN.AS'],
 ['CFEB.BR', 'PHARM.AS'],
 ['ASM.AS', 'PHIA.AS']]

In [76]:
pair1 = pairs[20][0]
pair2 = pairs[20][1]

std, mean = spread_stat(date3, date4, pair1, pair2)
df1, df2 = signal(date3, date4, std, mean, pair1, pair2)

IndexError: list index out of range

In [74]:
r2 = trading(df2)

In [75]:
r = trading(df1)

In [72]:
df1.to_csv('df.csv')

In [ ]:
df2.to_csv('dff.csv')